**NDVI**

In [17]:
from osgeo import gdal
from osgeo import ogr# 矢量处理数据包(shp,json等)
from osgeo import osr# 投影坐标包
from osgeo import gdal_array
from osgeo import gdalconst
import numpy as np

import json# json是存储和交换文本信息的语法，类似 XML
import requests# 网络数据获取(开源数据，不开源就用爬虫)
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def openRaster(fn,access=0):
  ds = gdal.Open(fn,access)
  if ds is None:
    print('error opening raster dataset')
  return ds

def getRasterBand(fn,band=1,access=0):
  ds = openRaster(fn,access)
  band = ds.GetRasterBand(band).ReadAsArray()
  return band

def createRasterFromcopy(fn,ds,data,driverFmt='GTiff'):
  driver = gdal.GetDriverByName(driverFmt)
  outds = driver.CreateCopy(fn,ds,strict=0)
  outds.GetRasterBand(1).WriteArray(data)
  ds = None
  outds = None

def createRasterFromTemplate(fn,ds,data,ndv=-9999.0,driverFmt='GTiff'):
  driver = gdal.GetDriverByName(driverFmt)
  outds = driver.Create(fn,xsize=ds.RasterXSize,ysize=ds.RasterYSize,bands=1,eType=gdal.GDT_Float32)
  outds.SetGeoTransform(ds.GetGeoTransform())
  outds.SetProjection(ds.GetProjection())
  outds.GetRasterBand(1).SetNoDataValue(ndv)
  outds.GetRasterBand(1).WriteArray(data)
  outds = None
  ds = None

def ndvi(nirband, redband, ndv=-9999.0):
  nirband[nirband < 0] = np.nan
  nirband[nirband > 10000] = np.nan
  redband[nirband < 0] = np.nan
  redband[nirband > 10000] = np.nan
  ndviband=(nirband-redband)/(nirband+redband)
  ndviband[np.isnan(ndviband)] = ndv
  return ndviband

infn = '/content/drive/MyDrive/AI data/GIS/landsat/subpatch.tif'
outfn = 'ndvi.tif'

# get file band
redband = getRasterBand(infn, 4).astype("float")
nirband = getRasterBand(infn, 3).astype("float")
# calculate ndvi band nalue
ndviband = ndvi(nirband,redband)
# create ndvi file
createRasterFromTemplate(outfn, gdal.Open(infn),ndviband)

**Reprojection a Raster with GDAL Warp**

In [ ]:
fn = '/content/drive/MyDrive/AI data/GIS/DEM/srtm_25_02.tif'
out = 'reprojection_4490.tif'
gdal.Warp(out,fn,dstSRS='EPSG:4490',width=6000,height=6000)
# gdal.Warp(out,fn,dstSRS='EPSG:4490',xRes=0.001,yRes=0.001)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f6def83bba0> >

**subset Raster**

In [ ]:
fn = '/content/drive/MyDrive/AI data/GIS/DEM/srtm_25_02.tif'
out = 'clip.tif'

# # example 1:top right corner stays the same
# ds = gdal.Open(fn)
# newRows = 700 # y coordinates
# newcols = 500 # x coordinate

# # get GeoTransform
# geot = ds.GetGeoTransform()
# # set clip raster shape
# driver = gdal.GetDriverByName('GTiff')
# ds_clip = driver.Create(out, xsize=newRows, ysize=newcols,bands=1,eType=gdal.GDT_Float32)
# # set clip GeoTransform
# ds_clip.SetGeoTransform(geot)
# # read ds value
# data = ds.GetRasterBand(1).ReadAsArray
# # set clip raster value
# ds_clip.GetRasterBand(1).WriteArray(data[:newRows,:newcols])
# # set clip raster nodata
# ds_clip.GetRasterBand(1).SetNoDataValue(ds.GetRasterBnad(1).GetNoDataValue())

# ds_clip = None

# # example 2:change top right corner stays the same
ds = gdal.Open(fn)
newRows = 500 # y coordinates
newcols = 500 # x coordinate

# get GeoTransform
geot = ds.GetGeoTransform()
print(geot)
print("geot[0]",geot[0])
print("geot[3]",geot[3])
# set clip raster shape
driver = gdal.GetDriverByName('GTiff')
ds_clip = driver.Create(out, xsize=newRows, ysize=newcols,bands=1,eType=gdal.GDT_Float32)

# 注意xoffset与yoffset值得设定
# 设定要参考GeoTransform中的pixelsize
# pixelsize乘上偏移像素就得到新文件得起始点位坐标
xoffset = 600*0.0008333333333333334 # meters=pixelsize * num_pixel
yoffset = 600*0.0008333333333333334 # meterss=pixelsize * num_pixel

# set new geotransform
geotnew = list(geot)
geotnew[0] = geotnew[0] + xoffset # x_origin + (pixelsize * num_pixel)
geotnew[3] = geotnew[3] - yoffset # y_origin - (pixelsize * num_pixel)

print("geotnew[0]:",geotnew[0])
print("geotnew[3]:",geotnew[3])

# set coordinates
ds_clip.SetGeoTransform(tuple(geotnew))
# set projection
ds_clip.SetProjection(ds.GetProjection())

startRow = round(yoffset/abs(geot[5])) # 返回浮点数 x 的四舍五入值,当参数n不存在时，round()函数的输出为整数。
startCol = round(xoffset/abs(geot[1])) # (pixelsize * num_pixel)/pixelsize
print("startRow:",startRow)
print("startCol:",startCol)

# read file_origin raster value
data = ds.GetRasterBand(1).ReadAsArray()
# set file_new raster value
ds_clip.GetRasterBand(1).WriteArray(data[startRow:startRow+newRows,startCol:startCol+newcols])
# set file_new raster nodata
ds_clip.GetRasterBand(1).SetNoDataValue(ds.GetRasterBnad(1).GetNoDataValue())

ds_clip = None

**Clip Raster to a polygon using gdal.Warp**

POLYGON--多边形面要素